In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

In [2]:
import sys
sys.path.append("./class")
from Transport_class import Transport

In [3]:
Nx=10
Ny=10
#phys_dims=[1.0,1.0]
phys_dims=[1.0,2.0]
g=pp.CartGrid([Nx,Ny],phys_dims)
#g=pp.CartGrid([Nx],phys_dims)
g.compute_geometry()            

In [4]:
unity = np.ones(g.num_cells)
empty = np.empty(0)
porosity=unity
aperture=1
bc_type=["dir","dir"]
bc_value=[0.,0.]
init_cond=lambda x,y,z:int(x<0.5)
print(g.num_faces)

220


In [5]:
specified_parameters = {
            "bc_type": bc_type,
            "bc_value": bc_value,
            "time_step": 0.01,
            "mass_weight": porosity * aperture,
            #"darcy_flux":np.zeros(g.num_faces),
            "darcy_flux":np.ones(g.num_faces),
            "t_max": 1,
            "method": "Explicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond,
}

In [6]:
specified_parameters["darcy_flux"][:(Nx+1)*Ny]=1.

In [7]:
transport=Transport(g,specified_parameters)

In [8]:
transport.set_bc()

In [9]:
tracer=np.zeros(g.num_cells)

In [10]:
transport.set_initial_cond(tracer)

In [11]:
data=transport.data[pp.PARAMETERS]["transport"]
kw_t="transport"
node_discretization = pp.Upwind(kw_t)
source_discretization = pp.ScalarSource(kw_t)
mass_discretization = pp.MassMatrix(kw_t)
        
node_discretization.discretize(transport.g,transport.data)
source_discretization.discretize(transport.g,transport.data)
mass_discretization.discretize(transport.g,transport.data)
        
A_upwind,b_upwind=node_discretization.assemble_matrix_rhs(transport.g,transport.data)
        #_,b_source=source_discretization.assemble_matrix_rhs(self.g,self.data)
A_mass,b_mass=mass_discretization.assemble_matrix_rhs(transport.g,transport.data)


In [12]:
print("A_mass:")
print(A_mass)


A_mass:
  (0, 0)	0.020000000000000004
  (1, 1)	0.020000000000000004
  (2, 2)	0.020000000000000007
  (3, 3)	0.019999999999999997
  (4, 4)	0.019999999999999997
  (5, 5)	0.020000000000000018
  (6, 6)	0.019999999999999997
  (7, 7)	0.019999999999999997
  (8, 8)	0.019999999999999997
  (9, 9)	0.019999999999999997
  (10, 10)	0.020000000000000004
  (11, 11)	0.020000000000000004
  (12, 12)	0.020000000000000007
  (13, 13)	0.019999999999999997
  (14, 14)	0.019999999999999997
  (15, 15)	0.020000000000000018
  (16, 16)	0.019999999999999997
  (17, 17)	0.019999999999999997
  (18, 18)	0.019999999999999997
  (19, 19)	0.019999999999999997
  (20, 20)	0.020000000000000007
  (21, 21)	0.02000000000000001
  (22, 22)	0.020000000000000014
  (23, 23)	0.020000000000000004
  (24, 24)	0.020000000000000004
  :	:
  (75, 75)	0.02000000000000001
  (76, 76)	0.01999999999999999
  (77, 77)	0.01999999999999999
  (78, 78)	0.01999999999999999
  (79, 79)	0.01999999999999999
  (80, 80)	0.019999999999999997
  (81, 81)	0.0199999

In [13]:
print("A_upwind:")
print(A_upwind)

A_upwind:
  (0, 0)	2.0
  (1, 1)	2.0
  (1, 0)	-1.0
  (2, 2)	2.0
  (2, 1)	-1.0
  (3, 3)	2.0
  (3, 2)	-1.0
  (4, 4)	2.0
  (4, 3)	-1.0
  (5, 5)	2.0
  (5, 4)	-1.0
  (6, 6)	2.0
  (6, 5)	-1.0
  (7, 7)	2.0
  (7, 6)	-1.0
  (8, 8)	2.0
  (8, 7)	-1.0
  (9, 9)	2.0
  (9, 8)	-1.0
  (10, 0)	-1.0
  (10, 10)	2.0
  (11, 1)	-1.0
  (11, 11)	2.0
  (11, 10)	-1.0
  (12, 2)	-1.0
  :	:
  (91, 90)	-1.0
  (92, 82)	-1.0
  (92, 92)	1.0
  (92, 91)	-1.0
  (93, 83)	-1.0
  (93, 93)	1.0
  (93, 92)	-1.0
  (94, 84)	-1.0
  (94, 94)	1.0
  (94, 93)	-1.0
  (95, 85)	-1.0
  (95, 95)	1.0
  (95, 94)	-1.0
  (96, 86)	-1.0
  (96, 96)	1.0
  (96, 95)	-1.0
  (97, 87)	-1.0
  (97, 97)	1.0
  (97, 96)	-1.0
  (98, 88)	-1.0
  (98, 98)	1.0
  (98, 97)	-1.0
  (99, 89)	-1.0
  (99, 99)	1.0
  (99, 98)	-1.0
